In [8]:
# MLflow Training
def train_elasticnet(N_alpha, N_rho):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)
    
    from data_processing import data_process

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        r2 = r2_score(actual, pred)
        return rmse, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the file
    try:
        df_raw = pd.read_csv('train.csv',index_col=0)
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)
        
    # Data processing.
    df_processed = data_process(df_raw)
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(df_processed)

    # The predicted column is "SalePrice" .
    train_x = train.drop(["SalePrice"], axis=1)
    test_x = test.drop(["SalePrice"], axis=1)
    train_y = train[["SalePrice"]]
    test_y = test[["SalePrice"]]

    # Set default values if no N_alpha is provided
    if int(N_alpha) is None:
        N_alpha = 50
    else:
        N_alpha = int(N_alpha)

    # Set default values if no N_rho is provided
    if int(N_rho) is None:
        N_rho = 11
    else:
        N_rho = int(N_rho)
    
    alphaRange = np.logspace(-3, -1, N_alpha)
    rhoRange   = np.linspace(0,0.2, N_rho) # we avoid very small rho by starting at 0.1
    scores     = np.zeros((N_rho, N_alpha))
      
    
    # Execute ElasticNet
    for alphaIdx, alpha in enumerate(alphaRange):
        for rhoIdx, rho in enumerate(rhoRange):
            with mlflow.start_run():
                lr = ElasticNet(alpha=alpha, l1_ratio=rho, normalize=False)
                lr.fit(train_x, train_y)
                train_r2 = lr.score(train_x, train_y)
        
        # Training Model Performances Evaluate Metrics
#         predicted_qualities = lr.predict(test_x)
#         (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)        

        # Evaluate Metrics
                predicted_qualities = lr.predict(test_x)
                (rmse, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
                print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, rho))
                print("  RMSE: %s" % rmse)
                print("  Train R2: %s" % train_r2)
                print("  Test R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
                mlflow.log_param("alpha", alpha)
                mlflow.log_param("l1_ratio", rho)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2", r2)
                #mlflow.log_metric("mae", mae)

                mlflow.sklearn.log_model(lr, "model")

In [2]:
# error plots test the assumptions - the same for linear regression 
# find the least rmse 


In [9]:
train_elasticnet(20,11)

Elasticnet model (alpha=0.001000, l1_ratio=0.000000):
  RMSE: 0.10999811398976517
  Train R2: 0.9553148149911568
  Test R2: 0.9290014713313135
Elasticnet model (alpha=0.001000, l1_ratio=0.020000):
  RMSE: 0.1087920643902138
  Train R2: 0.954679638962614
  Test R2: 0.9305498310419767
Elasticnet model (alpha=0.001000, l1_ratio=0.040000):
  RMSE: 0.10752661002194996
  Train R2: 0.953985492055752
  Test R2: 0.9321561040934572
Elasticnet model (alpha=0.001000, l1_ratio=0.060000):
  RMSE: 0.10659021515064263
  Train R2: 0.9532313981513244
  Test R2: 0.9333325953193231
Elasticnet model (alpha=0.001000, l1_ratio=0.080000):
  RMSE: 0.10613022464159019
  Train R2: 0.9525225705468687
  Test R2: 0.9339067606485351
Elasticnet model (alpha=0.001000, l1_ratio=0.100000):
  RMSE: 0.10594971563721643
  Train R2: 0.9518695817529661
  Test R2: 0.9341313956021566
Elasticnet model (alpha=0.001000, l1_ratio=0.120000):
  RMSE: 0.10586063015483799
  Train R2: 0.951228085898924
  Test R2: 0.9342421173610331
Ela

KeyboardInterrupt: 